In [25]:
# Import Numpy & PyTorch
import numpy as np
import torch

A tensor is a number, vector, matrix or any n-dimensional array.

## Problem Statement

We'll create a model that predicts crop yeilds for apples (*target variable*) by looking at the average temperature, rainfall and humidity (*input variables or features*) in different regions. 

Here's the training data:

>Temp | Rain | Humidity | Prediction
>--- | --- | --- | ---
> 73 | 67 | 43 | 56
> 91 | 88 | 64 | 81
> 87 | 134 | 58 | 119
> 102 | 43 | 37 | 22
> 69 | 96 | 70 | 103

In a **linear regression** model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yeild_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
```

It means that the yield of apples is a linear or planar function of the temperature, rainfall & humidity.



**Our objective**: Find a suitable set of *weights* and *biases* using the training data, to make accurate predictions.

## Training Data
The training data can be represented using 2 matrices (inputs and targets), each with one row per observation and one column for variable.

In [26]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [27]:
# Target (apples)
targets = np.array([[56], 
                    [81], 
                    [119], 
                    [22], 
                    [103]], dtype='float32')

Before we build a model, we need to convert inputs and targets to PyTorch tensors.

In [28]:
# Convert inputs and targets to tensors
inputs_tensor = torch.from_numpy(inputs)
targets_tensor = torch.from_numpy(targets)

print(inputs_tensor)
print(targets_tensor)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])


## Linear Regression Model (from scratch)

The *weights* and *biases* can also be represented as matrices, initialized with random values. The first row of `w` and the first element of `b` are use to predict the first target variable i.e. yield for apples, and similarly the second for oranges.

In [29]:
# Weights and biases
weights = torch.randn(1,3, requires_grad=True)
biases = torch.randn(1, requires_grad=True)
print(weights)
print(biases)

tensor([[0.1176, 0.3467, 1.0700]], requires_grad=True)
tensor([0.6651], requires_grad=True)


The *model* is simply a function that performs a matrix multiplication of the input `x` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

$$
\hspace{2.5cm} X \hspace{1.1cm} \times \hspace{1.2cm} W^T \hspace{1.2cm}  + \hspace{1cm} b \hspace{2cm}
$$

$$
\left[ \begin{array}{cc}
73 & 67 & 43 \\
91 & 88 & 64 \\
\vdots & \vdots & \vdots \\
69 & 96 & 70
\end{array} \right]
%
\times
%
\left[ \begin{array}{cc}
w_{11} & w_{21} \\
w_{12} & w_{22} \\
w_{13} & w_{23}
\end{array} \right]
%
+
%
\left[ \begin{array}{cc}
b_{1} & b_{2} \\
b_{1} & b_{2} \\
\vdots & \vdots \\
b_{1} & b_{2} \\
\end{array} \right]
$$

In [30]:
# Define the model
def model(x):
    return x @ weights.t() + biases

The matrix obtained by passing the input data to the model is a set of predictions for the target variables.

In [31]:
# Generate predictions
preds = model(inputs_tensor)

In [32]:
# Compare with targets
print("Predictions : ", preds)
print("Targets : ",targets_tensor)

Predictions :  tensor([[ 78.4879],
        [110.3549],
        [119.4124],
        [ 67.1582],
        [116.9609]], grad_fn=<AddBackward0>)
Targets :  tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])


Because we've started with random weights and biases, the model does not perform a good job of predicting the target varaibles.

## Loss Function

We can compare the predictions with the actual targets, using the following method: 
* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [33]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [34]:
# Compute loss
loss = mse(preds, targets_tensor)
print(loss)

tensor(720.3519, grad_fn=<DivBackward0>)


The resulting number is called the **loss**, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model.

## Compute Gradients

With PyTorch, we can automatically compute the gradient or derivative of the `loss` w.r.t. to the weights and biases, because they have `requires_grad` set to `True`.

More on autograd:  https://pytorch.org/docs/stable/autograd.html#module-torch.autograd

In [35]:
# Compute gradients
loss.backward()

In [36]:
# Gradients for weights
print(weights)
print(weights.grad)

tensor([[0.1176, 0.3467, 1.0700]], requires_grad=True)
tensor([[3967.2949, 2970.8953, 2207.0933]])


In [37]:
# Gradients for bias
print(biases)
print(biases.grad)

tensor([0.6651], requires_grad=True)
tensor([44.5498])


A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases. 

* If a gradient element is **postive**, 
    * **increasing** the element's value slightly will **increase** the loss.
    * **decreasing** the element's value slightly will **decrease** the loss.




* If a gradient element is **negative**,
    * **increasing** the element's value slightly will **decrease** the loss.
    * **decreasing** the element's value slightly will **increase** the loss.
    


The increase or decrease is proportional to the value of the gradient.

Finally, we'll reset the gradients to zero before moving forward, because PyTorch accumulates gradients.

In [38]:
weights.grad.zero_()
biases.grad.zero_()
print(weights.grad)
print(biases.grad)

tensor([[0., 0., 0.]])
tensor([0.])


## Adjust weights and biases using gradient descent

We'll reduce the loss and improve our model using the gradient descent algorithm, which has the following steps:

1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

In [39]:
# Generate predictions
preds = model(inputs_tensor)
print(preds)

tensor([[ 78.4879],
        [110.3549],
        [119.4124],
        [ 67.1582],
        [116.9609]], grad_fn=<AddBackward0>)


In [40]:
# Calculate the loss
loss = mse(preds, targets_tensor)
print(loss)

tensor(720.3519, grad_fn=<DivBackward0>)


In [41]:
# Compute gradients
loss.backward()

In [42]:
# Adjust weights & reset gradients
with torch.no_grad():
    weights -= weights.grad * 1e-5
    biases -= biases.grad * 1e-5
    weights.grad.zero_()
    biases.grad.zero_()

In [43]:
print(weights)

tensor([[0.0779, 0.3170, 1.0479]], requires_grad=True)


With the new weights and biases, the model should have a lower loss.

In [44]:
# Calculate loss
preds = model(inputs_tensor)
loss = mse(preds, targets_tensor)
print(loss)

tensor(477.3496, grad_fn=<DivBackward0>)


## Train for multiple epochs

To reduce the loss further, we repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch.

In [45]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs_tensor)
    loss = mse(preds, targets_tensor)
    loss.backward()
    with torch.no_grad():
        weights -= weights.grad * 1e-5
        biases -= biases.grad * 1e-5
        weights.grad.zero_()
        biases.grad.zero_()

In [46]:
# Calculate loss
preds = model(inputs_tensor)
loss = mse(preds, targets_tensor)
print(loss)

tensor(35.1976, grad_fn=<DivBackward0>)


In [47]:
# Print predictions
print(preds)

tensor([[ 58.4984],
        [ 85.6564],
        [108.7065],
        [ 28.4451],
        [103.7546]], grad_fn=<AddBackward0>)


In [48]:
# Print targets
print(targets)

[[ 56.]
 [ 81.]
 [119.]
 [ 22.]
 [103.]]
